In [1]:
import tensorflow as tf
# #指定使用那块GUP训练
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
config = tf.ConfigProto()
# 设置最大占有GPU不超过显存的70%
config.gpu_options.per_process_gpu_memory_fraction = 0.7 
# # 重点：设置动态分配GPU
config.gpu_options.allow_growth = True
# 创建session时
tf.Session(config=config) 

In [2]:
import numpy as np
from keras.datasets import cifar10
from torch.utils.data import Dataset
import torch.utils.data as Data
class MyDataset(Dataset):
    def __init__(self, imgs, transform=None):
        # super().__init__()
        self.imgs = imgs
        self.transform = transform

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, index):
        img = self.imgs[index]
        if self.transform is not None:
            img = self.transform(img)
        else:
            img = torch.from_numpy(img)
        img=img.reshape([3,32,32])
        return img
import torch
from torch.autograd import Variable
import model
import torch.nn.functional as F
model = model.cifar10(128)
model.load_state_dict(torch.load('./log/default/best-85.pth'))
model.cuda()
def get_mean_var(y_logit):
    y_logit=np.abs(y_logit-0.1)
    return np.mean(y_logit,axis=1)

def get_possibility(images):
    x_dataset = MyDataset(images)
    # print(x_dataset[0].shape)
    x_real_loader = Data.DataLoader(dataset=x_dataset, batch_size=100, shuffle=True)
    y_logits = []
    for i, data in enumerate(x_real_loader):
        # indx_target = target.clone()
        data = data.cuda()
        data = Variable(data, volatile=True)
        output = model(data)
        pred = F.softmax(output).cpu().detach().numpy()
        y_logits += [i for i in pred]
        y_logits=np.array(y_logits)
    return y_logits
import os
if not os.path.isdir('saved_models_{}'.format('bigan')):
    os.mkdir('saved_models_{}'.format('bigan'))
f = open('saved_models_{}/log_collapse1.txt'.format('bigan'), mode='w')
import torch.utils.data as Data
import cv2


from keras.datasets import cifar10
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, GaussianNoise
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers import MaxPooling2D, concatenate
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras import losses
from keras.utils import to_categorical
import keras.backend as K
import os
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

class BIGAN():
    def __init__(self):
        self.img_rows = 32
        self.img_cols = 32
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100
        self.x = []
        self.y = np.zeros((31, 1), dtype=np.int)
        self.y = list(self.y)
        for i in range(31):
            self.y[i] = []

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss=['binary_crossentropy'],
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # Build the encoder
        self.encoder = self.build_encoder()

        # The part of the bigan that trains the discriminator and encoder
        self.discriminator.trainable = False

        # Generate image from sampled noise
        z = Input(shape=(self.latent_dim, ))
        img_ = self.generator(z)

        # Encode image
        img = Input(shape=self.img_shape)
        z_ = self.encoder(img)

        # Latent -> img is fake, and img -> latent is valid
        fake = self.discriminator([z, img_])
        valid = self.discriminator([z_, img])

        # Set up and compile the combined model
        # Trains generator to fool the discriminator
        self.bigan_generator = Model([z, img], [fake, valid])
        self.bigan_generator.compile(loss=['binary_crossentropy', 'binary_crossentropy'],
            optimizer=optimizer)

    def build_encoder(self):
        model = Sequential()
        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        # model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(self.latent_dim))

        model.summary()
        img = Input(shape=self.img_shape)
        z = model(img)

        return Model(img, z)

    def build_generator(self):
        model = Sequential()

        model.add(Dense(128 * 8 * 8, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((8, 8, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()
        z = Input(shape=(self.latent_dim,))
        gen_img = model(z)

        return Model(z, gen_img)

    def build_discriminator(self):

        z = Input(shape=(self.latent_dim,))
        img = Input(shape=self.img_shape)
        d_in = concatenate([z, Flatten()(img)])

        model = Dense(32 * 32 * 3)(d_in)
        model = Reshape((32, 32, 3))(model)
        model = Conv2D(32, kernel_size=3, strides=2, padding="same")(model)
        model = LeakyReLU(alpha=0.2)(model)
        model = Dropout(0.25)(model)
        model = Conv2D(64, kernel_size=3, strides=2, padding="same")(model)
        model = ZeroPadding2D(padding=((0, 1), (0, 1)))(model)
        model = BatchNormalization(momentum=0.8)(model)
        model = LeakyReLU(alpha=0.2)(model)
        model = Dropout(0.25)(model)
        model = Conv2D(128, kernel_size=3, strides=2, padding="same")(model)
        model = BatchNormalization(momentum=0.8)(model)
        model = LeakyReLU(alpha=0.2)(model)
        model = Dropout(0.25)(model)
        model = Conv2D(256, kernel_size=3, strides=1, padding="same")(model)
        model = BatchNormalization(momentum=0.8)(model)
        model = LeakyReLU(alpha=0.2)(model)
        model = Dropout(0.25)(model)
        model = Flatten()(model)
        validity = Dense(1, activation="sigmoid")(model)

        return Model([z, img], validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (X_test, _) = cifar10.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_test = (X_test.astype(np.float32) - 127.5) / 127.5

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0
        steps = []
        values = []

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            # idx = np.random.randint(0, X_train.shape[0], batch_size)
            for index in range(nb_batches):
                global_step += 1
                # progress_bar.update(index)

                # get a batch of real images
                image_batch = X_train[index * batch_size:(index + 1) * batch_size]
                z = np.random.normal(size=(batch_size, self.latent_dim))
                imgs_ = self.generator.predict(z)

                # Select a random batch of images and encode
                # idx = np.random.randint(0, X_train.shape[0], batch_size)
                # imgs = X_train[idx]
                z_ = self.encoder.predict(image_batch)

                # Train the discriminator (img -> z is valid, z -> img is fake)
                d_loss_real = self.discriminator.train_on_batch([z_, image_batch], valid)
                d_loss_fake = self.discriminator.train_on_batch([z, imgs_], fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                # Train the generator (z -> img is valid and img -> z is is invalid)
                g_loss = self.bigan_generator.train_on_batch([z, image_batch], [valid, fake])

                # Plot the progress
                print("epoch:%d step:%d [D loss: %f, acc: %.2f%%] [G loss: %f]" % (epoch, global_step, d_loss[0],
                                                                                   100 * d_loss[1], g_loss[0]))

                if global_step % sample_interval == 0:
                    self.mode_drop(epoch, global_step)
                    

    def mode_drop(self, epoch, global_step):
        r, c = 10, 1000
        noise = np.random.normal(0, 1, (r * c, 100))
        # sampled_labels = np.array([num for _ in range(r) for num in range(c)])
        gen_imgs = self.generator.predict([noise])
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        y_logits = get_possibility(gen_imgs)
        metrics = get_mean_var(y_logits)


        f.writelines('\n')
        f.writelines('global_step:' + str(global_step))
        f.writelines('\n')
        f.writelines(' %.8f ' % (i) for i in metrics)
        f.writelines('\n')

if __name__ == '__main__':
    bigan = BIGAN()
    bigan.train(epochs=50, batch_size=64, sample_interval=200)


Using TensorFlow backend.
Sequential(
  (0): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (2): ReLU()
  (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (5): ReLU()
  (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (9): ReLU()
  (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (12): ReLU()
  (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (14): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 8192)              827392    
_________________________________________________________________
reshape_2 (Reshape)          (None, 8, 8, 128)         0         
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 16, 16, 128)       147584    
_________________________________________________________________
batch_normalization_4 (Batch (None, 16, 16, 128)       512       
_________________________________________________________________
activation_1 (Activation)    (None, 16, 16, 128)       0         
_________________________________________________________________
up_sampling2d_2 (UpSampling2 (None, 32, 32, 128)       0         
__________

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch:0 step:1 [D loss: 0.964462, acc: 37.50%] [G loss: 8.577759]
epoch:0 step:2 [D loss: 0.201850, acc: 90.62%] [G loss: 9.260620]
epoch:0 step:3 [D loss: 0.096309, acc: 96.88%] [G loss: 8.171257]
epoch:0 step:4 [D loss: 0.126112, acc: 96.88%] [G loss: 7.489373]
epoch:0 step:5 [D loss: 0.074383, acc: 97.66%] [G loss: 7.229850]
epoch:0 step:6 [D loss: 0.238366, acc: 89.06%] [G loss: 9.996765]
epoch:0 step:7 [D loss: 1.133828, acc: 52.34%] [G loss: 13.831806]
epoch:0 step:8 [D loss: 0.682707, acc: 87.50%] [G loss: 12.374109]
epoch:0 step:9 [D loss: 0.466773, acc: 88.28%] [G loss: 9.828083]
epoch:0 step:10 [D loss: 0.063870, acc: 98.44%] [G loss: 7.730422]
epoch:0 step:11 [D loss: 0.094764, acc: 98.44%] [G loss: 7.158998]
epoch:0 step:12 [D loss: 0.203022, acc: 92.97%] [G loss: 8.476723]
epoch:0 step:13 [D loss: 0.309770, acc: 89.84%] [G loss: 6.754344]
epoch:0 step:14 [D loss: 0.076388, acc: 96.88%] [G loss: 6.062761]
epoch:0 step:15 [D loss: 0.054439, acc: 98.44%] [G loss: 6.016621]
ep

epoch:0 step:124 [D loss: 0.104793, acc: 96.88%] [G loss: 4.176692]
epoch:0 step:125 [D loss: 0.047242, acc: 99.22%] [G loss: 4.208336]
epoch:0 step:126 [D loss: 0.080829, acc: 97.66%] [G loss: 5.052876]
epoch:0 step:127 [D loss: 0.113743, acc: 97.66%] [G loss: 4.748745]
epoch:0 step:128 [D loss: 0.308571, acc: 86.72%] [G loss: 7.432187]
epoch:0 step:129 [D loss: 0.085200, acc: 98.44%] [G loss: 7.713721]
epoch:0 step:130 [D loss: 0.034524, acc: 100.00%] [G loss: 6.342452]
epoch:0 step:131 [D loss: 0.134647, acc: 95.31%] [G loss: 5.987045]
epoch:0 step:132 [D loss: 0.622276, acc: 80.47%] [G loss: 9.078184]
epoch:0 step:133 [D loss: 0.035613, acc: 99.22%] [G loss: 10.728908]
epoch:0 step:134 [D loss: 0.039746, acc: 96.88%] [G loss: 9.740320]
epoch:0 step:135 [D loss: 0.064414, acc: 99.22%] [G loss: 5.785068]
epoch:0 step:136 [D loss: 0.137718, acc: 94.53%] [G loss: 6.378257]
epoch:0 step:137 [D loss: 0.114570, acc: 93.75%] [G loss: 6.301042]
epoch:0 step:138 [D loss: 0.325369, acc: 85.16

...

epoch:49 step:38575 [D loss: 0.025884, acc: 100.00%] [G loss: 5.194890]
epoch:49 step:38576 [D loss: 0.191215, acc: 91.41%] [G loss: 9.608160]
epoch:49 step:38577 [D loss: 0.239392, acc: 95.31%] [G loss: 6.318770]
epoch:49 step:38578 [D loss: 0.657505, acc: 59.38%] [G loss: 7.425192]
epoch:49 step:38579 [D loss: 0.577597, acc: 59.38%] [G loss: 7.012256]
epoch:49 step:38580 [D loss: 0.133444, acc: 98.44%] [G loss: 2.119492]
epoch:49 step:38581 [D loss: 0.074942, acc: 99.22%] [G loss: 6.730690]
epoch:49 step:38582 [D loss: 0.230523, acc: 97.66%] [G loss: 5.256300]
epoch:49 step:38583 [D loss: 0.049135, acc: 100.00%] [G loss: 9.436772]
epoch:49 step:38584 [D loss: 0.026430, acc: 100.00%] [G loss: 10.490608]
epoch:49 step:38585 [D loss: 0.066667, acc: 100.00%] [G loss: 6.424169]
epoch:49 step:38586 [D loss: 0.017244, acc: 100.00%] [G loss: 5.904021]
epoch:49 step:38587 [D loss: 0.136035, acc: 96.88%] [G loss: 5.236854]
epoch:49 step:38588 [D loss: 0.093387, acc: 99.22%] [G loss: 4.354608]


epoch:49 step:38692 [D loss: 0.343118, acc: 78.91%] [G loss: 7.580604]
epoch:49 step:38693 [D loss: 0.252813, acc: 95.31%] [G loss: 6.377961]
epoch:49 step:38694 [D loss: 0.483327, acc: 65.62%] [G loss: 5.415696]
epoch:49 step:38695 [D loss: 0.145349, acc: 99.22%] [G loss: 7.394864]
epoch:49 step:38696 [D loss: 0.369601, acc: 80.47%] [G loss: 7.502138]
epoch:49 step:38697 [D loss: 0.223315, acc: 99.22%] [G loss: 5.546886]
epoch:49 step:38698 [D loss: 0.046620, acc: 100.00%] [G loss: 8.641668]
epoch:49 step:38699 [D loss: 0.125207, acc: 97.66%] [G loss: 7.010818]
epoch:49 step:38700 [D loss: 0.199984, acc: 95.31%] [G loss: 6.430087]
epoch:49 step:38701 [D loss: 0.204497, acc: 96.88%] [G loss: 3.743399]
epoch:49 step:38702 [D loss: 0.048427, acc: 100.00%] [G loss: 6.744344]
epoch:49 step:38703 [D loss: 0.128434, acc: 99.22%] [G loss: 12.118643]
epoch:49 step:38704 [D loss: 0.032855, acc: 100.00%] [G loss: 8.645916]
epoch:49 step:38705 [D loss: 0.135494, acc: 97.66%] [G loss: 6.320517]
ep

epoch:49 step:38808 [D loss: 0.670615, acc: 53.91%] [G loss: 8.384217]
epoch:49 step:38809 [D loss: 0.017148, acc: 100.00%] [G loss: 8.949610]
epoch:49 step:38810 [D loss: 0.264538, acc: 96.09%] [G loss: 5.280114]
epoch:49 step:38811 [D loss: 0.084229, acc: 100.00%] [G loss: 9.455040]
epoch:49 step:38812 [D loss: 0.205056, acc: 94.53%] [G loss: 6.403120]
epoch:49 step:38813 [D loss: 0.107993, acc: 99.22%] [G loss: 8.787320]
epoch:49 step:38814 [D loss: 0.053095, acc: 100.00%] [G loss: 9.230204]
epoch:49 step:38815 [D loss: 0.293014, acc: 88.28%] [G loss: 8.225316]
epoch:49 step:38816 [D loss: 0.152877, acc: 99.22%] [G loss: 4.497293]
epoch:49 step:38817 [D loss: 0.099332, acc: 99.22%] [G loss: 5.722535]
epoch:49 step:38818 [D loss: 0.161037, acc: 97.66%] [G loss: 5.880548]
epoch:49 step:38819 [D loss: 0.270448, acc: 90.62%] [G loss: 3.789473]
epoch:49 step:38820 [D loss: 0.494071, acc: 71.09%] [G loss: 6.251849]
epoch:49 step:38821 [D loss: 0.240785, acc: 92.97%] [G loss: 5.904859]
epo

epoch:49 step:38925 [D loss: 0.303494, acc: 90.62%] [G loss: 4.952621]
epoch:49 step:38926 [D loss: 0.127383, acc: 98.44%] [G loss: 9.160003]
epoch:49 step:38927 [D loss: 0.936235, acc: 50.00%] [G loss: 9.354731]
epoch:49 step:38928 [D loss: 0.771654, acc: 55.47%] [G loss: 4.544422]
epoch:49 step:38929 [D loss: 0.027309, acc: 100.00%] [G loss: 2.632430]
epoch:49 step:38930 [D loss: 0.420621, acc: 83.59%] [G loss: 6.801983]
epoch:49 step:38931 [D loss: 0.214916, acc: 93.75%] [G loss: 5.513331]
epoch:49 step:38932 [D loss: 0.248527, acc: 92.97%] [G loss: 8.068051]
epoch:49 step:38933 [D loss: 0.018493, acc: 100.00%] [G loss: 8.807262]
epoch:49 step:38934 [D loss: 0.148823, acc: 96.09%] [G loss: 8.185399]
epoch:49 step:38935 [D loss: 0.265561, acc: 91.41%] [G loss: 5.257712]
epoch:49 step:38936 [D loss: 0.076732, acc: 100.00%] [G loss: 7.524589]
epoch:49 step:38937 [D loss: 0.523497, acc: 67.97%] [G loss: 5.425125]
epoch:49 step:38938 [D loss: 0.122027, acc: 98.44%] [G loss: 8.406659]
epo

epoch:49 step:39043 [D loss: 0.110067, acc: 99.22%] [G loss: 7.953777]
epoch:49 step:39044 [D loss: 0.405203, acc: 82.03%] [G loss: 8.449113]
epoch:49 step:39045 [D loss: 0.173005, acc: 96.88%] [G loss: 9.083858]
epoch:49 step:39046 [D loss: 0.366549, acc: 92.19%] [G loss: 6.602138]
epoch:49 step:39047 [D loss: 0.859794, acc: 50.78%] [G loss: 6.354909]
epoch:49 step:39048 [D loss: 0.139893, acc: 100.00%] [G loss: 7.053217]
epoch:49 step:39049 [D loss: 0.334944, acc: 80.47%] [G loss: 7.338583]
epoch:49 step:39050 [D loss: 0.507283, acc: 73.44%] [G loss: 5.112873]
